<a href="https://colab.research.google.com/github/OoMyGit/Data-Mining/blob/main/%5BAnswer%5D_Exercise_Week11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kwandy Chandra - 0706022210001
## Week 11 - Data Mining (A)

In [30]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder

!pip install mlxtend==0.23.1

# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here:
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [31]:
# load the data set ans show the first five transaction
url = 'https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv'
df = pd.read_csv(url, delimiter = ',')
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased

---




In [32]:
unique_products = set(df.values.flatten())

print(unique_products)

{nan, 'Eggs', 'Milk', 'Wine', 'Pencil', 'Bagel', 'Bread', 'Diaper', 'Meat', 'Cheese'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [33]:
#create an itemset based on the products
itemset = {item: 0 for item in unique_products}

# encoding the feature
for item in df.iloc[0]:
    if item in itemset:
        itemset[item] = 1

itemset

{nan: 0,
 'Eggs': 1,
 'Milk': 0,
 'Wine': 1,
 'Pencil': 1,
 'Bagel': 0,
 'Bread': 1,
 'Diaper': 1,
 'Meat': 1,
 'Cheese': 1}

In [34]:
# create new dataframe from the encoded features
encoded_df = pd.DataFrame(0, index=range(len(df)), columns=itemset)

# Encode each transaction
for i, row in df.iterrows():
    for item in row:
        encoded_df.loc[i, item] = 1

# show the new dataframe
encoded_df.head()

,NaN,Eggs,Milk,Wine,Pencil,Bagel,Bread,Diaper,Meat,Cheese
0,0,1,0,1,1,0,1,1,1,1
1,0,0,1,1,1,0,1,1,1,1
2,1,1,1,1,0,0,0,0,1,1
3,1,1,1,1,0,0,0,0,1,1
4,1,0,0,1,1,0,0,0,1,0


In [37]:
# Since, the encoded dataframe consist of the empty column. We will drop the NaN column or u can use the index.
# Select all columns except the first one
encoded_df = encoded_df.iloc[:, 1:]
encoded_df.head()

,Eggs,Milk,Wine,Pencil,Bagel,Bread,Diaper,Meat,Cheese
0,1,0,1,1,0,1,1,1,1
1,0,1,1,1,0,1,1,1,1
2,1,1,1,0,0,0,0,1,1
3,1,1,1,0,0,0,0,1,1
4,0,0,1,1,0,0,0,1,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products.
For this case study, we will min_support=0.2

In [40]:
#Set threshold value untuk digunakan dalam penghitungan support
from mlxtend.frequent_patterns import apriori, association_rules

# Apply Apriori algorithm to find frequent itemsets with min_support=0.2
frequent_itemsets = apriori(encoded_df, min_support=0.2, use_colnames=True)

# Show the result
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.438095,(Eggs)
1,0.501587,(Milk)
2,0.438095,(Wine)
3,0.361905,(Pencil)
4,0.425397,(Bagel)
5,0.504762,(Bread)
6,0.406349,(Diaper)
7,0.476190,(Meat)
8,0.501587,(Cheese)
9,0.244444,"(Eggs, Milk)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [41]:
# Generate association rules with confidence threshold of 0.6
association_rules_df = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

# Show the association rules
association_rules_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624,0.387409
1,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203,0.469167
2,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148,0.350053
3,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148,0.350053
4,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754,0.330409
5,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650,0.402687
6,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891,0.526414
7,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540,0.500891
8,"(Eggs, Cheese)",(Meat),0.298413,0.476190,0.215873,0.723404,1.519149,0.073772,1.893773,0.487091
9,"(Eggs, Meat)",(Cheese),0.266667,0.501587,0.215873,0.809524,1.613924,0.082116,2.616667,0.518717


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__, __conviction__, __conviction__ and the interpretation from the case above (please use text section)

# Kwandy Chandra - 0706022210001



## **Metrics**


1.   **Antecedent Support**

 This tells us how frequently the antecedent (the item(s) that appear before the arrow in the rule) occurs in the dataset.

2.   **Consequent Support**

  This tells us how frequently the consequent (the item(s) that appear after the arrow in the rule) occurs in the dataset.


3.   **Support**

  This measures how often the items in the rule (both antecedent and consequent) appear together in the dataset.

4.   **Confidence**

  This indicates the strength of the rule; higher confidence means the rule is more likely to hold true in the data.

5.   **Lift**

  A lift value greater than 1 indicates that the antecedent and consequent are positively correlated (i.e., they occur together more often than by chance). A lift value less than 1 indicates a negative correlation.

6.   **Leverage**

  Leverage values close to 0 suggest that the antecedent and consequent are independent. Positive leverage values indicate a stronger relationship, while negative leverage values suggest a negative relationship.

7.   **Conviction**

  A higher conviction value indicates a stronger rule. A conviction of 1 means that the antecedent and consequent are independent.

8.   **Zhang's Metric**

  Zhang’s metric typically ranges between 0 and 1, where higher values indicate stronger association.


## **Interpretation of Results**


*   **High Confidence**: Rules like (Eggs) -> (Meat) and (Milk) -> (Cheese) have high confidence values (greater than 0.6), indicating strong associations. For example, for (Eggs) -> (Meat), the confidence is 0.608696, which suggests that when "Eggs" are purchased, there's a 60.87% chance that "Meat" will be bought as well.


*   **Lift Greater than 1**: Lift values such as 1.278261 (for (Eggs) -> (Meat)) indicate that the purchase of "Eggs" increases the likelihood of purchasing "Meat" beyond what would be expected by chance.


*   **Leverage**: The leverage values show that there is a positive relationship between products like "Eggs" and "Meat". For example, the leverage of 0.058050 indicates that "Eggs" and "Meat" are purchased together more often than by random chance.


*   **Conviction**: Conviction values (e.g., 1.338624 for (Eggs) -> (Meat)) suggest that these items are likely to be associated, with values significantly higher than 1 indicating a positive relationship.


*   **Zhang’s Metric**: This new metric suggests a reasonable association strength between items like "Eggs" and "Meat" (0.387409), further corroborating the confidence and lift metrics.


## **Summary**


These metrics help in evaluating the strength of relationships between products in a market basket. By analyzing these values, businesses can identify strong associations between products and make informed decisions, such as cross-selling or creating promotions based on frequently purchased item combinations.